# Voorbeeld van omzetting door Veg2Hab

Dit notebook doorloopt alle stappen die tot nu toe geimplementeerd zijn voor Veg2Hab. Het is een voorbeeld van hoe de omzetting van vegetatie naar habitattypekaart kan verlopen.

Eerst worden alle benodigde resources geinstantieerd (was-wordt lijst, definitietabel, fgr-kaart). Hierna wordt de omzetting stap voor stap uitgevoerd.

In [1]:
from pathlib import Path

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_waswordtlijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
from veg2hab.io.cli import CLIInterface
import pandas as pd
from veg2hab.bronnen import FGR, Bodemkaart, LBK

pd.set_option('display.max_columns', 100)

CLIInterface.get_instance()

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Was-Wordt lijst opschonen en inladen

VvN en SBB codes worden gecheckt op validiteit bij:
1. het opschonen van een was-wordt lijst
2. bij het inlezen van een opgeschoonde was-wordt lijst

In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_waswordtlijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)
wwl.df.head(3)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide


,VvN,SBB
0,14rg10,14-l
1,None,7-a
2,None,8b-b


### Definitietabel opschonen en inladen

VvN- en SBB-codes worden gecheckt op validiteit bij:
1. het opschonen van een definitietabel
2. bij het inlezen van een opgeschoonde definitietabel

In [3]:
path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.json")
path_in_mozaiekjson = Path("../data/mozaiekjson.json")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_in_mozaiekjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)
dt.df.head()

Alle SBB codes zijn valide
Alle VvN codes zijn valide


AttributeError: 'NoneType' object has no attribute 'as_letter'

AttributeError: 'NoneType' object has no attribute 'as_letter'

## Inladen van een kartering vanaf een shapefile

In [4]:
shp_path = Path("../testing/vegetatiekarteringen/GR/SGL Hunzedal en Leekstermeer2021/2021 Vegetatiekartering Leekstermeer2021/GIS bestanden Onlanden 2021/Vegetatiekartering_Leekstermeer2021.shp")
Kartering.from_shapefile(shp_path, vegtype_col_format="single", ElmID_col="elmid", sbb_of_vvn="sbb", SBB_col="SBBTYPE", split_char="+", lok_vegtypen_col="Vegtype").gdf.head(5)

,ElmID,_LokVegTyp,geometry,SBBTYPE,SBBTYPE1,SBBTYPE2,SBBTYPE3,Datum,Opmerking,Opp,_LokVrtNar,perc_0,perc_1,perc_2,VegTypeInfo
0,144,C3a+F1b,"POLYGON ((225536.382 578550.118, 225536.613 57...",08C-a+16-a,8c-a,16-a,None,None,None,1692.139818,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['8c-a'], VvN: []), (50.0%, SBB:..."
1,143,B3c+C2c,"POLYGON ((225518.346 578389.608, 225514.596 57...",08-a+08C2c,8-a,8c2c,None,None,None,2974.448552,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['8-a'], VvN: []), (50.0%, SBB: ..."
2,73,F1b,"POLYGON ((225398.991 578372.192, 225399.792 57...",16-a,16-a,None,None,None,None,4582.870987,Lokale typologie is primair vertaald naar sbb,100.0,0.0,0.0,"[(100.0%, SBB: ['16-a'], VvN: [])]"
3,62,H1a+A1,"POLYGON ((225338.120 578506.385, 225330.908 57...",12B1d+50A,12b1d,50a,None,None,None,2843.780077,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['12b1d'], VvN: []), (50.0%, SBB..."
4,111,A1+B1,"POLYGON ((225731.271 578748.007, 225730.075 57...",50A+08B3d,50a,8b3d,None,None,None,1911.539469,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['50a'], VvN: []), (50.0%, SBB: ..."


## Inladen en omzetten van een access kartering

- Voor ieder vlak in de kartering wordt de (complexe) SBB code opgezocht.
- Deze SBB-codes worden opgeschoond en gevalideerd.
- De SBB codes worden toegevoegd aan de dataframe van de kartering als VegTypeInfo instance.
  
De resulterende dataframe heeft voor elke regel in KarteringVegetatietype.csv een VegTypeInfo instance in de rij van de betreffende shape.

In [5]:
shape_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/vlakken.shp")
csvs_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/864_RottigeMeenthe2013.mdb")
shape_elm_id_column = "ElmID"

access_kartering = Kartering.from_access_db(shape_path, shape_elm_id_column, csvs_path)

access_kartering.gdf.head(3)


mdbtools v1.0.0


,ElmID,geometry,Opmerking,Datum,Opp,_LokVrtNar,intern_id,Locatie_x,VegTypeInfo,Locatie_y,_LokVegTyp
0,28049,"POLYGON ((189433.818 538314.302, 189439.215 53...",None,None,3761.736281,Lokale typologie is primair vertaald naar SBB,1809,1809,"[(90%, SBB: ['9a2a'], VvN: []), (10%, SBB: ['9...",1809,"09a2-6 (90%), 09a2-1 (10%)"
1,15034,"POLYGON ((190568.500 539171.122, 190538.023 53...",None,None,10442.658431,Lokale typologie is primair vertaald naar SBB,815,815,"[(100%, SBB: ['50a'], VvN: [])]",815,50a-1 (100%)
2,13503,"POLYGON ((189538.104 539409.332, 189532.399 53...",None,None,2174.098680,Lokale typologie is primair vertaald naar SBB,650,650,"[(90%, SBB: ['9a2a'], VvN: []), (10%, SBB: ['9...",650,"09a2-1 (90%), 09a2-6 (10%)"


### Inladen bronnen

In [6]:
mask = access_kartering.get_geometry_mask()
fgr = FGR(Path("../data/bronbestanden/FGR.json"))
bodemkaart = Bodemkaart(Path("../testing/bronbestanden/Bodemkaart.gpkg"), mask=mask)
lbk = LBK(Path("../testing/bronbestanden/LBK_2023_Beta.shp"), mask=mask)

#### Opzoeken en toevoegen VvN aan kartering

In [7]:
access_kartering.apply_wwl(wwl)
# Voorbeeld van 3 vegtypeinfos in 1 geometry
access_kartering.gdf["VegTypeInfo"].iloc[3]

[VegTypeInfo(percentage=80, SBB=[SBB(klasse='9', verbond='b', associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='a')], VvN=[]),
 VegTypeInfo(percentage=10, SBB=[SBB(klasse='9', verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='g')], VvN=[VvN(klasse='9', orde=None, verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='3')]),
 VegTypeInfo(percentage=10, SBB=[SBB(klasse='9', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None)], VvN=[VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None)])]

#### Vinden van de mogelijke habitattypen voor een VegTypeInfo

Voor elke VegTypeInfo worden de mogelijke habitattypen opgezocht in de definitietabel. Deze worden vervolgens in een HabitatVoorstel gezet, samen met de code waarop de match is gemaakt, de regel in de definitietabel die is gebruikt en het niveau van de match.

In [8]:
access_kartering.apply_deftabel(dt)
access_kartering.gdf["HabitatVoorstel"]

0       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1       [[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...
2       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
3       [[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...
4       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
                              ...                        
1944    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1945    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1946    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1947    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1948    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
Name: HabitatVoorstel, Length: 1949, dtype: object

In [9]:
# Voorbeeld met voor 2 VegTypeInfo's waarbij de eerste twee mogelijke voorstellen heeft
access_kartering.gdf["HabitatVoorstel"][45]

[[HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='39', orde='a', verbond='a', associatie='1', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='39', orde='a', verbond='a', associatie='1', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), habtype='H2180_B', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=171, mits=FGRCriterium(fgrtype=<FGRType.DU: 'Duinen'>), mozaiek=GeenMozaiekregel(mozaiek_threshold=95), match_level=6, mozaiek_dict=None)],
 [HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), habtype='H2180_B', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=178, mits=FGRCriterium(fgrtype=<FGRType.DU: 'Duinen'>), mozaiek=GeenMozaiekregel(mozaiek_threshold=95), match_level=6, mozaiek

### De mitsen en mozaiekregels checken en de HabitatVoorstellen omzetten naar HabitatKeuzes

In [10]:
access_kartering.bepaal_habitatkeuzes(fgr, bodemkaart, lbk)


Iteratie 0: van 3499 naar 104 keuzes nog te bepalen
Iteratie 1: van 104 naar 34 keuzes nog te bepalen
Iteratie 2: van 34 naar 29 keuzes nog te bepalen
Iteratie 3: van 29 naar 29 keuzes nog te bepalen


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:1091: UserWarning: Er zijn nog 29 habitatkeuzes die niet bepaald konden worden.
  f"Er zijn nog {n_keuzes_still_to_determine_post} habitatkeuzes die niet bepaald konden worden."


In [11]:
access_kartering.gdf.head()

,ElmID,geometry,Opmerking,Datum,Opp,_LokVrtNar,intern_id,Locatie_x,VegTypeInfo,Locatie_y,_LokVegTyp,HabitatVoorstel,HabitatKeuze,finished_on_iteration
0,28049,"POLYGON ((189433.818 538314.302, 189439.215 53...",None,None,3761.736281,Lokale typologie is primair vertaald naar SBB,1809,1809,"[(90%, SBB: ['9a2a'], VvN: ['9aa2a']), (10%, S...",1809,"09a2-6 (90%), 09a2-1 (10%)",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.DUIDELIJK: 1...,1
1,15034,"POLYGON ((190568.500 539171.122, 190538.023 53...",None,None,10442.658431,Lokale typologie is primair vertaald naar SBB,815,815,"[(100%, SBB: ['50a'], VvN: [])]",815,50a-1 (100%),[[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...,[HabitatKeuze(status=<KeuzeStatus.NIET_GEAUTOM...,1
2,13503,"POLYGON ((189538.104 539409.332, 189532.399 53...",None,None,2174.098680,Lokale typologie is primair vertaald naar SBB,650,650,"[(90%, SBB: ['9a2a'], VvN: ['9aa2a']), (10%, S...",650,"09a2-1 (90%), 09a2-6 (10%)",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.DUIDELIJK: 1...,1
3,15075,"POLYGON ((190801.313 539561.740, 190819.617 53...",None,None,18878.050297,Lokale typologie is primair vertaald naar SBB,2515,2515,"[(80%, SBB: ['9b-a'], VvN: []), (10%, SBB: ['9...",2515,"09b-4 (80%), 09-13 (10%), 09a2-2 (10%)",[[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...,[HabitatKeuze(status=<KeuzeStatus.GEEN_KLOPPEN...,1
4,20502,"POLYGON ((190497.231 537724.945, 190505.062 53...",None,None,4950.611630,Lokale typologie is primair vertaald naar SBB,1316,1316,"[(58%, SBB: ['8b3c'], VvN: ['8bb4d']), (29%, S...",1316,"08b3-3 (58%), 08c-6 (29%), 09-11 (10%), 05d5-1...",[[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...,[HabitatKeuze(status=<KeuzeStatus.VEGTYPEN_NIE...,1


### De kartering formatten als een Habitattypenkartering

Dit pakt de HabitatVoorstellen uit en format het geheel als een Habitattypenkartering zoals beschreven in Gegevens Leverings Protocol (uitvraag bijlage 3a).

Voor nu checken we nog geen mitsen en mozaiek en wordt voor ieder VegTypeInfo het eerste HabitatVoorstel gebruikt.

In [12]:
final_format = access_kartering.as_final_format()
final_format

,Area,Opm,Datum,ElmID,geometry,_Samnvttng,_LokVegTyp,_LokVrtNar,Habtype1,Perc1,Opp1,Kwal1,Opm1,SBB1,VvN1,_VgTypInf1,_Status1,_Uitleg1,_SBBdftbl1,_VvNdftbl1,_Mits_opm1,_Mozk_opm1,_MozkPerc1,Habtype2,Perc2,Opp2,Kwal2,Opm2,SBB2,VvN2,_VgTypInf2,_Status2,_Uitleg2,_SBBdftbl2,_VvNdftbl2,_Mits_opm2,_Mozk_opm2,_MozkPerc2,Habtype3,Perc3,Opp3,Kwal3,Opm3,SBB3,VvN3,_VgTypInf3,_Status3,_Uitleg3,_SBBdftbl3,_VvNdftbl3,_Mits_opm3,_Mozk_opm3,_MozkPerc3,Habtype4,Perc4,Opp4,Kwal4,Opm4,SBB4,VvN4,_VgTypInf4,_Status4,_Uitleg4,_SBBdftbl4,_VvNdftbl4,_Mits_opm4,_Mozk_opm4,_MozkPerc4,Habtype5,Perc5,Opp5,Kwal5,Opm5,SBB5,VvN5,_VgTypInf5,_Status5,_Uitleg5,_SBBdftbl5,_VvNdftbl5,_Mits_opm5,_Mozk_opm5,_MozkPerc5
0,3761.736281,None,None,28049,"POLYGON ((189433.818 538314.302, 189439.215 53...",100.0% H7140_B,"09a2-6 (90%), 09a2-1 (10%)",Lokale typologie is primair vertaald naar SBB,H7140_B,90,338556,G,Er is een duidelijke keuze. Kloppende mits en ...,None,9aa2a,"(90%, SBB: ['9a2a'], VvN: ['9aa2a'])",KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,None,"['9aa2', 566, 'H7140_B']","Mits: Geen mits (altijd waar), TRUE","Mozaiekregel: Geen mozaiekregel (altijd waar),...",,H7140_B,10,37617.4,G,Er is een duidelijke keuze. Kloppende mits en ...,None,9aa2a,"(10%, SBB: ['9a2a'], VvN: ['9aa2a'])",KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,None,"['9aa2', 566, 'H7140_B']","Mits: Geen mits (altijd waar), TRUE","Mozaiekregel: Geen mozaiekregel (altijd waar),...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10442.658431,None,None,15034,"POLYGON ((190568.500 539171.122, 190538.023 53...",100.0% HXXXX,50a-1 (100%),Lokale typologie is primair vertaald naar SBB,HXXXX,100,1.04427e+06,X,Er zijn mitsen of mozaiekregels die nog niet g...,50a,None,"(100%, SBB: ['50a'], VvN: [])",KeuzeStatus.NIET_GEAUTOMATISEERD_CRITERIUM,Er zijn niet-geautomatiseerde mitsen/mozaiekre...,"['50a', 585, 'H7220']",None,"Mitsen: [['50a', 'H7220', '(Niet geautomatisee...","Mozaiekregels: [['50a', 'H7220', 'Geen mozaiek...",5.96% zelfstandig HXXXX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2174.098680,None,None,13503,"POLYGON ((189538.104 539409.332, 189532.399 53...",100.0% H7140_B,"09a2-1 (90%), 09a2-6 (10%)",Lokale typologie is primair vertaald naar SBB,H7140_B,90,195669,G,Er is een duidelijke keuze. Kloppende mits en ...,None,9aa2a,"(90%, SBB: ['9a2a'], VvN: ['9aa2a'])",KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,None,"['9aa2', 566, 'H7140_B']","Mits: Geen mits (altijd waar), TRUE","Mozaiekregel: Geen mozaiekregel (altijd waar),...",,H7140_B,10,21741,G,Er is een duidelijke keuze. Kloppende mits en ...,None,9aa2a,"(10%, SBB: ['9a2a'], VvN: ['9aa2a'])",KeuzeStatus.DUIDELIJK,Als alle regels gevolgd worden is er 1 duideli...,None,"['9aa2', 566, 'H7140_B']","Mits: Geen mits (altijd waar), TRUE","Mozaiekregel: Geen mozaiekregel (altijd waar),...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18878.050297,None,None,15075,"POLYGON ((190801.313 539561.740, 190819.617 53...","80.0% H0000, 10.0% H7140_B, 10.0% HXXXX","09b-4 (80%), 09-13 (10%), 09a2-2 (10%)",Lokale typologie is primair vertaald naar SBB,HXXXX,10,188781,X,Er zijn mitsen of mozaiekregels die nog niet g...,None,9rg3,"(10%, SBB: ['9-g'], VvN: ['9rg3'])",KeuzeStatus.NIET_GEAUTOMATISEERD_CRITERIUM,Er zijn niet-geautomatiseerde mitsen/mozaiekre...,None,"['9rg3', 230, 'H2190_C']","Mitsen: [['9rg3', 'H2190_C', '(FGR is Noordzee...","Mozaiekregels: [['9rg3', 'H2190_C', 'Geen moza...",72.70% zelfstandig HXXXX,H7140_

In [13]:
access_kartering.final_format_to_file(Path("../testing/veg2hab_output.shp"))

In [14]:
g = access_kartering.gdf
g[g.ElmID==11003].iloc[0].HabitatVoorstel

[[HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), habtype='H2180_B', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=178, mits=FGRCriterium(fgrtype=<FGRType.DU: 'Duinen'>), mozaiek=GeenMozaiekregel(mozaiek_threshold=95), match_level=6, mozaiek_dict=defaultdict(<class 'int'>, {})),
  HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie='a', derivaatgemeenschap=None, rompgemeenschap=None), vegtype_in_dt=VvN(klasse='40', orde='a', verbond='a', associatie='2', subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None), habtype='H91D0', kwaliteit=<Kwaliteit.GOED: 'Goed'>, idx_in_dt=650, mits=EnCriteria(sub_criteria=[NietCriterium(sub_criterium=FGRCriterium(fgrtype=<FGRType.DU: 'Duine